# Automation to create several Reports in a loop !

In [16]:
import pickle
from script_utils import Experiment
from script_utils import DataProcessor as DP
import os

import pandas as pd
import numpy as np

import matplotlib.backends.backend_pdf as pdf_backend
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [17]:
def create_charts_to_pdf(df, title, metadata, filename):
    base_features = set()

    for column in df.columns:
        base_feature = column.replace('_pred', '')
        base_features.add(base_feature)

    with pdf_backend.PdfPages(filename) as pdf:
        # Add metadata to the beginning of the PDF
        fig = plt.figure(figsize=(6, 10))
        plt.title('Test Metadata')
        plt.figtext(0.5, 0.05, metadata, ha='center', fontsize=8)
        #plt.figtext(0.1, 0.9, metadata)  # Adjust the coordinates as needed
        plt.axis('off')
        pdf.savefig(fig)
        plt.close(fig)

        MAE = {}
        MSE = {}
        for base_feature in base_features:
            pred_feature = f'{base_feature}_pred'

            fig = plt.figure(figsize=(10, 6))
            plt.plot(df.index, df[base_feature], label=base_feature, color='#1f77b4')
            plt.plot(df.index, df[pred_feature], label=pred_feature, color='#2ca02c')

            plt.xlabel('Time')
            plt.ylabel(base_feature)
            plt.title(f'{title}')
            plt.legend()

            # Calculate metrics
            if base_feature.startswith("WDIR"):
                mae = DP.mean_absolute_error_WDIR(df[base_feature], df[pred_feature])
                mse = DP.mean_square_error_WDIR(df[base_feature], df[pred_feature])
            else:
                mae = mean_absolute_error(df[base_feature], df[pred_feature])
                mse = mean_squared_error(df[base_feature], df[pred_feature])
            MAE[base_feature] = mae.round(3)
            MSE[base_feature] = mse.round(3)

            # Add metrics to the current page of the PDF
            fig.text(0.01, 0.02, f"MAE: {mae.round(3)}\nMSE: {mse.round(3)}")  # Adjust the coordinates as needed
            pdf.savefig(fig)
            plt.close(fig)

        return MAE, MSE

In [18]:
def create_error_table(dictionary):
    # Extract unique measurements and station IDs
    measurements = set()
    station_ids = set()
    for key in dictionary.keys():
        measurement, station_id, *_ = key.split('_')
        measurements.add(measurement)
        station_ids.add(station_id)

    # Create empty table
    table = pd.DataFrame(index=sorted(station_ids), columns=sorted(measurements))

    # Fill table with dictionary values
    for key, value in dictionary.items():
        measurement, station_id, *_ = key.split('_')
        if key.endswith('ERA5'):
            measurement += '_ERA5'
        table.loc[station_id, measurement] = value

    # Replace missing values with NaN
    table = table.fillna(np.nan)

    return table

In [19]:
#Type = True for MLM and false for SSUM
def evaluation(filename, type):

    # open the file for reading in binary mode
    report: Experiment
    with open(f'data/reports/Publication/center_42003/{filename}', 'rb') as f:
        # load the object from the file using pickle.load()
        report = pickle.load(f)

    #Create directory if not exists
    filename = f"{filename.replace('.pickle','')}"
    directory = f"data/evaluation/Publication/center_42003//{filename}"
    if not os.path.exists(directory):
        os.makedirs(directory)  # Create the directory

   #Create PDF report
    title =  f"One-Shot-Forecast of: {filename.replace('.pickle','')}"
    MAE_dict, MSE_dict = create_charts_to_pdf(report.one_shot_forecast, title, report.get_settings_string(), f"{directory}/{filename}_charts.pdf")


    if type:
        MAE_df = create_error_table(MAE_dict)
        MAE_df.to_csv(f"{directory}/{filename}_MAE.csv")
        MSE_df = create_error_table(MSE_dict)
        MSE_df.to_csv(f"{directory}/{filename}_MSE.csv")

    else:
        MAE_df = pd.DataFrame(MAE_dict.items(), columns=['Feature', 'MAE'])
        MSE_df = pd.DataFrame(MSE_dict.items(), columns=['Feature', 'MSE'])
        error_df = pd.merge(MAE_df, MSE_df, on='Feature')
        error_df.to_csv(f"{directory}/{filename}_Error.csv", index=False)


    print(f"Saved: {directory}/{filename}")

In [20]:
approach = "MLM"   # "MLM" ... Multi Location Modelling, "SSUM" ... Station Specific Unified Model
model_names = ["LSTM_2", "CNN_2", 'transformer']    #["LSTM", "GRU", "CNN", "TCN"]
alpha_values = np.arange(0, 1.1, 0.1)   #lower boundary, upper boundary, increment

if approach == "MLM":
    for model in model_names:
        for alpha in alpha_values:
            filename = f'report_A_{model}_{format(alpha, ".1f")}.pickle'
            evaluation(filename, True)

elif approach == "SSUM":
    for model in model_names:
        for alpha in alpha_values:
            filename = f'report_B_{model}_{format(alpha, ".1f")}.pickle'
            evaluation(filename, False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/reports/Publication/center_42003/report_A_LSTM_2_0.0.pickle'

# Create Specific reports:

In [20]:
filenames = ["report_A_LSTM_2_0.7.pickle",
             "report_A_CNN_2_0.2.pickle",
             "report_A_transformer_0.2.pickle",
            ]
for filename in filenames:
    evaluation(filename, True)

Saved: data/evaluation/Publication/Stage_2/report_A_LSTM_2_0.7/report_A_LSTM_2_0.7
Saved: data/evaluation/Publication/Stage_2/report_A_CNN_2_0.2/report_A_CNN_2_0.2
Saved: data/evaluation/Publication/Stage_2/report_A_transformer_0.2/report_A_transformer_0.2


# Single Report:

In [22]:
evaluation('report_42003_stage0_LSTM_1.pickle', True)

Saved: data/evaluation/Publication/center_42003//report_42003_stage0_LSTM_1/report_42003_stage0_LSTM_1
